In [1]:
import os

In [2]:
os.chdir('../')


In [3]:
%pwd

'f:\\ProjectAI\\ChatSystem'

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
api_key = os.getenv("GEMINI_API_KEY")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str 
    local_data_file: Path
    augmentation: bool

In [7]:
from ChatBoxSystem.constants import *
from ChatBoxSystem.utils.helper import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            augmentation=self.params.data_ingestion.augmentation
        )

        return data_ingestion_config

In [9]:
import os
import gdown
from ChatBoxSystem import logger
import pandas as pd
from google import genai
from google.genai import types

In [18]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix + file_id, zip_download_dir)

            logger.info(f"Downloaded file from :[{dataset_url}] and saved at :[{zip_download_dir}]")
        except Exception as e:
            raise e


    def data_augmentation(self):
        if self.config.augmentation:
            df = pd.read_csv(self.config.local_data_file)
            augmented_records = []
            for _, row in df.iterrows():
                paraphrases = self.paraphrase_text(row['Answer'])
                for p in paraphrases:  # nếu paraphrases là list
                    augmented_records.append({
                        "Question": row['Question'],
                        "Answer": p
                    })
            df_augmented = pd.DataFrame(augmented_records)
            df_augmented.to_csv(self.config.local_data_file, index=False)
            logger.info(f"Data augmentation completed and saved to {self.config.local_data_file}")
        else:
            logger.info("Data augmentation is disabled.")


    def paraphrase_text(self, text: str) -> list:
        prompt = f"Paraphrase the following text into 5 different ways:\n{text}"
        client = genai.Client(api_key=api_key)

        response = client.models.generate_content(
            model="gemini-2.5-flash",  # hoặc gemini-2 nếu bạn có quyền
            contents=prompt
        )

        # response.output_text chứa kết quả text
        paraphrases = [line.strip() for line in response.text.split("\n") if line.strip()]
        return paraphrases

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.data_augmentation()
    logger.info("Data ingestion completed successfully.")
except Exception as e:
    logger.exception(e)
    raise e

[2025-10-01 17:55:33,408: INFO: helper]: YAML file config\config.yaml loaded successfully.
[2025-10-01 17:55:33,409: INFO: helper]: YAML file params.yaml loaded successfully.
[2025-10-01 17:55:33,410: INFO: helper]: Directory created at: artifacts
[2025-10-01 17:55:33,411: INFO: helper]: Directory created at: artifacts/data_ingestion


Downloading...
From: https://drive.google.com/uc?/export=download&id=1AixdvjgwB0np9dIrxVJrmlzWI6AaDhwq
To: f:\ProjectAI\ChatSystem\artifacts\data_ingestion\data.csv
100%|██████████| 2.94k/2.94k [00:00<?, ?B/s]

[2025-10-01 17:55:35,613: INFO: 1440940172]: Downloaded file from :[https://drive.google.com/file/d/1AixdvjgwB0np9dIrxVJrmlzWI6AaDhwq/view?usp=sharing] and saved at :[artifacts/data_ingestion/data.csv]
[2025-10-01 17:55:35,645: INFO: models]: AFC is enabled with max remote calls: 10.


[2025-10-01 17:55:45,600: INFO: _client]: HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
[2025-10-01 17:55:45,621: INFO: models]: AFC is enabled with max remote calls: 10.
[2025-10-01 17:55:54,584: INFO: _client]: HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
[2025-10-01 17:55:54,608: INFO: models]: AFC is enabled with max remote calls: 10.
[2025-10-01 17:56:02,135: INFO: _client]: HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
[2025-10-01 17:56:02,160: INFO: models]: AFC is enabled with max remote calls: 10.
[2025-10-01 17:56:08,754: INFO: _client]: HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
[2025-10-01 17:56:08,776: INFO: models]: AFC is enabled with max remote calls: 